Connected to PRL (Python 3.11.0)

In [3]:
for gamma in (0.94, 0.95, 0.96):
    for num_layers in (5,6):
        print(f'\n\n Layers: {num_layers}, Gamma: {gamma}')

        seed = 2705
        rep = 2000000
        batch_size = 32
        #gamma = 0.96
        epsilon = 1.0
        epsilon_decay = 99999
        epsilon_min = 0.1
        learning_rate = 5e-5
        price_horizon = 48
        future_horizon = 0
        hidden_dim = 96
        #num_layers = 5
        positions = False
        action_classes = 3
        reward_shaping = True
        factor = 1
        verbose = False
        TRAIN = True
        df = train_name



        if TRAIN:

            # Initialize Environment
            env = Electric_Car(path_to_test_data=df)


            #Initialize DQN
            agent = DDQNAgent(env = env,
                            features = features_train,
                            epsilon_decay = epsilon_decay,
                            epsilon_start = epsilon,
                            epsilon_end = epsilon_min,
                            discount_rate = gamma,
                            lr = learning_rate,
                            buffer_size = 100000,
                            price_horizon = price_horizon,
                            hidden_dim=hidden_dim,
                            num_layers = num_layers,
                            positions = positions,
                            action_classes = action_classes, 
                            verbose = verbose)

            obs, r, terminated, _, _ = env.step(random.randint(-1,1)) # Reset environment and get initial observation
            state, grads = agent.obs_to_state(obs)

            
            episode_balance = 0
            episode_loss = 0
            episode_counter = 0
            episode_reward = 0

            with tqdm(total=rep) as pbar:
                for i in range(rep):

                    action = agent.choose_action(i, state, greedy = False) # Choose action (discrete)
                    cont_action = agent.action_to_cont(action) # Convert to continuous action
                    
                    new_obs, r, t, _, _ = env.step(cont_action)
                    new_state, new_grads = agent.obs_to_state(new_obs)
                    
                    # Reward Shaping            
                    new_reward = agent.shape_reward(r, cont_action, grads, apply = reward_shaping, factor = factor)

                    # Fill replay buffer - THIS IS THE ONLY THING WE DO WITH THE CURRENT OBSERVATION - LEARNING IS FULLY PERFORMED FROM THE REPLAY BUFFER
                    if state.shape[0] == agent.state_dim and new_state.shape[0] == agent.state_dim:
                        agent.replay_memory.add_data((state, action, new_reward, t, new_state))

                    #Update DQN
                    loss = agent.optimize(batch_size)
                    
                    # Update values
                    episode_balance += r
                    episode_reward += r
                    episode_loss += loss

                    # New observation
                    state = new_state
                    grads = new_grads # Gradients for reward shaping
                    
                    pbar.update(1)

                    if t:
                        # Reset Environment
                        env.counter = 0
                        env.hour = 1
                        env.day = 1
                        episode_counter += 1
                        print('Episode ', episode_counter, 'Balance: ', episode_balance, 'Reward: ', episode_reward, 'Loss: ', episode_loss) # Add both balance and reward to see how training objective and actually spent money differ
                        episode_loss = 0
                        episode_balance = 0
                        episode_reward = 0
                        
                        
                        if episode_counter % 4 == 0:
                            # Evaluate DQN
                            train_dqn = DDQNEvaluation(price_horizon = price_horizon)
                            train_dqn.evaluate(agent = agent)
                            
                            # Reset Environment
                            env.counter = 0
                            env.hour = 1
                            env.day = 1
                            

            # Save agent
            torch.save(agent.dqn_predict.state_dict(), f'models/agent_layers{num_layers}_gamma{gamma}.pt')
            pbar.close



 Layers: 5, Gamma: 0.94


/Users/maxfeucht/Documents/VU/Project Reinforcement Learning/BatteryGrid/agent.py:88: RuntimeWarning: invalid value encountered in divide
  return (var - np.min(var)) / (np.max(var) - np.min(var))


Number of engineered features:  57
State dimension:  113
Number of DQN Parameters:  39171


  1%|▏         | 25281/2000000 [01:20<1:35:05, 346.09it/s]

Episode  1 Balance:  -7784.130952777774 Reward:  -7784.130952777774 Loss:  1960.3349933915306


...

Absolute Balance:  -340.87299672839515


  7%|▋         | 130501/2000000 [06:42<1:37:38, 319.09it/s]

Episode  5 Balance:  -1315.5047007407393 Reward:  -1315.5047007407393 Loss:  760.4706829024944


  8%|▊         | 152305/2000000 [08:13<1:39:47, 308.58it/s]


KeyboardInterrupt: 